In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# for utils
import scipy
from scipy.spatial.transform import Rotation as Rot

In [ ]:
# utils
def plot_poses(Trs, enum=False):
    xs = []
    ys = []
    plt.plot(0, 0, marker='o', markersize=20, c='r', linestyle='None', label='Origin')
    for index, Tr in enumerate(Trs):
        x = Tr[0,2]
        y = Tr[1,2]
        R3 = np.eye(3)
        R3[:2,:2] = Tr[:2,:2]
        t = Rot.from_matrix(R3).as_rotvec()[2]
        
        plt.plot(x, y, marker=(3, 0, 180*t/np.pi-90), markersize=20, linestyle='None')
        plt.plot(x, y, marker=(2, 0, 180*t/np.pi-90), markersize=30, linestyle='None')
        if enum:
            plt.text(x, y, index, size=20)
        xs.append(x)
        ys.append(y)
    plt.plot(xs, ys)    
    plt.legend()
    plt.axis('equal')
    plt.grid()
    
def gTl_is_correct():
    x_ = np.random.randn(3)
    x_[2] = np.clip(x_[2], -np.pi/2.1, +np.pi/2.1)
    res = gTl(x_)
    assert res.shape == (3,3), "T must be 3x3 matrix"
    
    mat = np.eye(3); mat[0,2] = x_[0]; mat[1,2] = x_[1]
    if 'as_matrix' in dir(Rot):
        mat[:2,:2] = Rot.from_rotvec([0,0,x_[2]]).as_matrix()[:2,:2]
    else:
        mat[:2,:2] = Rot.from_rotvec([0,0,x_[2]]).as_dcm()[:2,:2]
    return np.linalg.norm(np.linalg.inv(mat) @ res - np.eye(3)) < 1e-14

# I. RBT in 2D

There are three major uses for a transformation matrix $T$.  

__Use (a): to represent the configuration (position and orientation) of a rigid body;__

__Use (b): to change the reference frame in which a vector is represented;__

__Use (c): to transform a reference frame.__  


Suppose, the origin of a robot (= origin of robot's ref. frame) is located in the point $(x,y)$ of a global frame   
and robot's frame is rotated by angle $\theta$:

<img src="./global_local.png">

Its pose (pose = position + orientation) can be represented by the vector $$\mathrm x = \begin{bmatrix} x\\y\\ \theta \end{bmatrix}$$

or by the transformation matrix:
$$^gT_l(\mathrm x)$$

_Here $^gT_l$ means that local frame is described wrt. global frame_

### Task 1.A (already solved for you)
Code the function that from a XYT pose generates a RBT matrix.

In [ ]:
# T(x)
# use numpy only
def gTl(state):
    def c(t):
        return np.cos(t)
    def s(t):
        return np.sin(t)
    x, y, t = state
    return np.array([
        [c(t), -s(t), x],
        [s(t), c(t), y],
        [0, 0, 1]
    ])

In [ ]:
# Check out
gTl_is_correct()

In [ ]:
# Robot at the origin of global frame
x = [0,0,0]
T_origin = gTl(x)
T_origin

In [ ]:
# plot
Ts = [T_origin]
plot_poses(Ts)

In [ ]:
# Purely rotated robot
x = [0,0,np.pi/6]
# your code

In [ ]:
# plot
Ts = [T_rot]
plot_poses(Ts)

In [ ]:
# Purely translated robot
x = [5,4,0]
#your code

In [ ]:
# plot
Ts = [T_transl]
plot_poses(Ts)

<br/>
<br/>
<br/>

__Use (b): to change the reference frame in which a vector or frame is represented;__  

Suppose, the robot observes some landmark $p$ in position $^lp = \begin{bmatrix} ^l x \\ ^l y \\ 1 \end{bmatrix}$ of __robot's__ frame.  
The robot must send global coordinates $^gp$ of $p$ to computational module that process global coordinates of landmarks and plots a map of landmarks. 
<img src="./vector.png">

### Task 1.B.

Write down the formula to get $^gp$ with known $^lp$ and robot's pose $^gT_l$.  
Provide numerical result for given $^lp$ and $^gT_l$.

In [ ]:
# Given data
lp = [1.0, 1.3, 1]
T = gTl([0.4, 0.5, 1.2])

# code

__Use (c): to transform a reference frame.__  

Suppose, the robot has current pose $^gT_a$ wrt. global frame.  
Then the robot updates its pose by $^aT_b$ and the updated state is $^gT_b$.
<img src="./delta.png">

### Task 1.C.
Obtain $^gT_b$ given pose $a$ and transformation from $a$ to $b$.

In [ ]:
state_a = [0.3,0.5,0]
atob = [0.3, -0.4, -np.pi/6]

In [ ]:
gTa = gTl(state_a)
gTa

In [ ]:
# plot
Ts = [gTa]
plot_poses(Ts)

In [ ]:
# your code
aTb = ...
aTb

In [ ]:
# your code
gTb = ...
gTb

In [ ]:
# plot
Ts = [gTa, gTb]
plot_poses(Ts)

## On the inverse of a RBT (optional)

Sometimes there is need to describe global frame wrt. local frame.  
Imagine robot computing the way back home that is located in the origin of global frame.  

<img src="./inverse.png">

This can be simply done by inverse transformation $$^lT_g = (^gT_l)^{-1}.$$

### Task 2.
This is a theoretical task. Obtain and write down the inverse transformation.
Use the internal structure of the transformation matrix:
$$^gT_l = \begin{bmatrix} R & t \\ 0^\top & 1 \end{bmatrix},$$
where the rotation matrix $R$ and displacement $t$ are known.

_Trick: better to:_
- do block multiplication of $^gT_l \, ^lT_g  = I$:
$$\begin{bmatrix} R & t \\ 0^\top & 1 \end{bmatrix} \cdot \begin{bmatrix} M & a \\ 0^\top & 1 \end{bmatrix} = I$$
- express unknown $M$ and $a$ through $R$ and $t$
- assemble $T^{-1}$ by $M$ and $a$

__$^gT_l^{-1}(R,t)$ must not have any matrix inverse inside!__

1.

$\begin{bmatrix} R & t \\ 0^\top & 1 \end{bmatrix} \cdot \begin{bmatrix} M & a \\ 0^\top & 1 \end{bmatrix} = $

$\begin{bmatrix} RM + t \cdot 0^\top & Ra + t\cdot1 \\ 0^\top M + 1 \cdot 0^\top & 0^\top a + 1 \cdot 1 \end{bmatrix} = $

$\begin{bmatrix} RM & Ra + t \\ 0^\top & 1 \end{bmatrix} = I$

2. 
$RM = ?$

$RM = I_{2x2}$

$Ra + t = ?$

$Ra + t = 0$

$M = R^{-1} = R^\top$

$Ra = - t$

$R^{-1}Ra = - R^{-1}t$

$a = - R^\top t$

__Answer__
$$^lT_g = (^gT_l)^{-1} = \begin{bmatrix} R^\top & - R^\top t \\ 0^\top & 1 \end{bmatrix}.$$

# II. RBT in 3D

In [ ]:
import mrob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Plot preparations: 

In [ ]:
def plotConfig():
    "configfures the 3d plot structure for representing tranformations"
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    return ax
    
def plotT(T, ax):
    "Plots a 3 axis frame in the origin given the mrob SE3 transformation, right-hand convention"
    # transform 3 axis to the coordinate system
    x = np.zeros((4,3))
    x[0,:] = T.transform(np.array([0,0,0], dtype='float64'))
    x[1,:] = T.transform(np.array([1,0,0], dtype='float64'))
    ax.plot(x[[0,1],0],x[[0,1],1],x[[0,1],2],'r') # X axis
    x[2,:] = T.transform(np.array([0,1,0], dtype='float64'))
    ax.plot(x[[0,2],0],x[[0,2],1],x[[0,2],2],'g') # Y axis
    x[3,:] = T.transform(np.array([0,0,1], dtype='float64'))
    ax.plot(x[[0,3],0],x[[0,3],1],x[[0,3],2],'b') # Z axis
    plt.xlabel('x')
    plt.ylabel('y')

## Create rotation vector and map: 
$$\Large \mathbb{R}^3 \to SO(3)$$
We will use the Exponent map (call by the constructor SO3), from a random 3D vector. Check the norm.

In [ ]:
rotation_angles = np.random.rand(3)
print(rotation_angles)
R = mrob.geometry.SO3(rotation_angles)
# Just in case you are using an old version of the library
#with mrob.ostream_redirect(stdout=True, stderr=True):
R

### Properties:Check inverse

In [ ]:
# your code from np. The np array is at R.R()

In [ ]:
R.inv()

## Relation to Rotations in 2D embeded in 3D

In [ ]:
angle = 0.2
rotation_angles = np.array([0,0,angle])
print(rotation_angles)
R = ...
print('Rotation: \n', R)
# From previous function
xyt = np.array([0,0,angle])
T = ...
print(T[:2,:2])

## Logarithm
$$\Large SO(3)\to \mathbb{R}^3$$
The second operation is the logarithm .Ln(), check that the mapping provides a vector

In [ ]:
theta = R.Ln()
print(theta)

## Create translation vector and build RBT matrix: 
$$\Large \begin{equation*}
T = 
\begin{bmatrix}
R & t \\
0 & 1
\end{bmatrix}
\end{equation*}$$

In [ ]:
translation_vector = np.array([5, 5, 0])
T = mrob.geometry.SE3(R,translation_vector)
print(T)
ax = plotConfig()
plotT(T, ax)
plt.title('Rotation from bare-hands')
plt.show()

## Create rotation + translation vector
$$\Large \begin{equation*}
\xi = 
\begin{bmatrix}
\theta_1 \\
\theta_2 \\
\theta_3 \\
\rho_1 \\
\rho_2 \\
\rho_3
\end{bmatrix}
\end{equation*}$$

### Create RBT matrix, i.e. this mapping:
$$\Large \rm I\!R^6 \rightarrow \rm SE(3)$$
### Inside the .SE3() method, this formula is applied:
$$\Large T = Exp(\xi)$$

In [ ]:
T1 = mrob.geometry.SE3(np.concatenate([rotation_angles, translation_vector]))
T1

### We see, that the translation in T2 doesn't exactly correspond to translation_vector.

### RBT composition
Multiply two RBT and see the result. You can compare with the method multiply from the SE3 objects. Check out the order.

In [ ]:
T1 = mrob.geometry.SE3(mrob.geometry.SO3(np.array([0,0,0.15])), np.array([5,0,0]))
print(T1)
T2 = mrob.geometry.SE3(mrob.geometry.SO3(np.array([0,0,0.25])), np.array([2,3,0]))
print(T2)
# code

## 2.B Distances between Rotations and RBT
Calculate the distance between two rotations by:
 * Calculating directly by the equation $d(R_1,R_2) = Log(R_1 * R_2^{\top})$
 * Calculate the ditance with the function .distance()

In [ ]:
R1 = mrob.geometry.SO3(np.array([0,0,-0.2]))
R2 = mrob.geometry.SO3(np.array([0,0,0.3]))
print(R1)
print(R2)

In [ ]:
# your code

In [ ]:
# your code